In [1]:
from harp.clock import decode_harp_clock
from beh_functions import session_dirs, parseSessionID, load_model_dv
from open_ephys.analysis import Session
import datetime
from aind_ephys_rig_qc.temporal_alignment import search_harp_line
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import os
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from scipy.io import loadmat
from scipy.stats import zscore

from open_ephys.analysis import Session
from pathlib import Path
import glob

import spikeinterface as si
import xarray as xr
import json
import seaborn as sns
from PyPDF2 import PdfMerger
from aind_ephys_utils import align 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import re

%matplotlib inline
%matplotlib widget

In [2]:
session_id = 'behavior_716325_2024-05-31_10-31-14'
model_name = 'stan_qLearning_5params'
session_dir = session_dirs(session_id, model_name)

In [3]:
model_dv = load_model_dv(session_dir)

AttributeError: 'NoneType' object has no attribute 'to_dataframe'